In [2]:

#testing geopy 
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


from geopy.geocoders import Nominatim
geolocator = Nominatim()
a=(-56, 3.21346208441)
location = geolocator.reverse(a, language='en')
print(location.address)

Bouvet Island, Norway


In [3]:
#retrieving country name
print(location.address.split(",")[-1])

 Norway


In [99]:
#Original csv from http://hdr.undp.org/en/data




#testing pandas
import pandas as pd
import math
uData = pd.read_csv('Total unemployment rate (% of labour force).csv',names=['HDI_rank', 'Country', '1995', '2000', '2005', '2010', '2011', '2012', '2013', '2014', '2015'])
#remove 2 first rows with no data
uData = df.ix[2:]


In [98]:
#creating big table with all the coordenates we need
#math.cos(1*math.pi/180) degrees to  radians
#Чтобы найти для любой параллели длину дуги в 1°, нужно умножить 111,3 км (длину дуги экватора в 1°) на косинус угла, соответствующего географической широте

# lengh of  1° of circle of latitude: 111.3 km * cos(angle between latitude and equator)

# I modified it to make points for one latitude - because getting data from geopy takes so much time - it is done per 1 circle of latitude
def coordsAlloc(latitude):
    kmperpoint=200
    result = []
    pointsHs = kmperpoint/math.cos(latitude*math.pi/180)/111.3
    result.append((latitude,0))
    pcounter=pointsHs
    while pcounter<180:
        result.append((latitude,pcounter))
        result.append((latitude,-1*pcounter))
        pcounter += pointsHs
    return result


In [6]:
#main data mining function
import time
def assignCountries(df):
    for i in df.index:
        time.sleep(0.60)#geopy really likes to time-out and drop me
        location = geolocator.reverse(df.ix[i]['Coords'], language='en', timeout=1111)
        if location.address!=None:
            #print location.address
            df.ix[i]['Country']=location.address.split(",")[-1]
            
#assignCountries(coordsTable)
#MegaTable=coordsTable

In [80]:

#coordsTable
#looong procedure trying to  get data from geopy
# it takes 3-4 minites per 1 degree of latitude! - or server would cut me for "too many requests"
for i in range(-90,90):
    tmpTable = pd.DataFrame(columns=['Coords','Country'])
    tmpTable = pd.DataFrame({'Coords':coordsAlloc(i)})
    tmpTable['Country']=''
    assignCountries(tmpTable)
    MegaTable=pd.concat([MegaTable,tmpTable])
    time.sleep(20.50)
    #cleaning...
    MegaTable=MegaTable[MegaTable.Country != '']
    MegaTable=MegaTable.reset_index(drop=True)
    
    #this is for me to check progress while I play Battlefield
    print("////////////")
    print(i)
    print(len(MegaTable.index))
    #saving results  - various connection errors are common
    MegaTable.to_csv('partialMEGAtable.csv', index=False, header=False, encoding='utf-8')


////////////
69
9654
////////////
70
9707
////////////
71
9752
////////////
72
9786
////////////
73
9813
////////////
74
9841
////////////
75
9868
////////////
76
9889
////////////
77
9910
////////////
78
9926
////////////
79
9940
////////////
80
9951
////////////
81
9962
////////////
82
9973
////////////
83
9978
////////////
84
9983
////////////
85
9984
////////////
86
9986
////////////
87
9986
////////////
88
9986
////////////
89
9987


In [87]:
#Cleaning and saving table
MegaTable=MegaTable.drop_duplicates()

MegaTable=MegaTable[MegaTable.Country != '']
MegaTable=MegaTable.reset_index(drop=True) #recalculate index
#len(MegaTable[MegaTable.duplicated()]) #duplicates check
#MegaTable[MegaTable["Coords"]==str((-37, -11.2500957606))]
#len(MegaTable.drop_duplicates())
MegaTable.to_csv('partialMEGAtable.csv', index=False, header=False, encoding='utf-8')
#MegaTable.ix[9986]["Country"]#[1:-1].split(",")


u'N'

In [163]:
#restoring MEGA TABLE TO ITS FORMER GLORY
MegaTable= pd.read_csv('partialMEGAtable.csv',names=['Coords', 'Country'])
#df
#pd.set_option('display.max_rows', 60)
#IMPORTANT - after saving and loading from file the table - first and second rows are now strings - was tuple and unicode 
#aus = MegaTable.ix[666]["Country"]
#aus in "Australia"
#uData.ix[124]['2000']

#drop territorial warets from dataframe 
MegaTable=MegaTable[MegaTable.Country.str.contains("erritor")==False]


#lets add colums to main table
MegaTable['1995']=''
MegaTable['1995']=''
MegaTable['2000']=''
MegaTable['2005']=''
MegaTable['2010']=''
MegaTable['2011']=''
MegaTable['2012']=''
MegaTable['2013']=''
MegaTable['2014']=''
MegaTable['2015']=''

In [169]:
#aggregating all data


#df - MegaTable, df2 - uData
def fillYears(df,df2):
    for elementMT in df.index:
        countryMT = df.ix[elementMT]["Country"]
        for elementUD in df2.index:
            countryUD= str(df2.ix[elementUD]["Country"])
            if countryUD[0]==" ": countryUD=countryUD[1:] #spacebar cleaning
            if (countryUD in countryMT) or (countryMT in countryUD):
                df.ix[elementMT]["1995"]=df2.ix[elementUD]["1995"]
                df.ix[elementMT]["2000"]=df2.ix[elementUD]["2000"]
                df.ix[elementMT]["2005"]=df2.ix[elementUD]["2005"]
                df.ix[elementMT]["2010"]=df2.ix[elementUD]["2010"]
                df.ix[elementMT]["2011"]=df2.ix[elementUD]["2011"]
                df.ix[elementMT]["2012"]=df2.ix[elementUD]["2012"]
                df.ix[elementMT]["2013"]=df2.ix[elementUD]["2013"]
                df.ix[elementMT]["2014"]=df2.ix[elementUD]["2014"]
                df.ix[elementMT]["2015"]=df2.ix[elementUD]["2015"]
                    
    


In [178]:
#about 5 minutes of work - but we cant directly merge dataframes because country data isnt clean
fillYears(MegaTable,uData)


#lets remove rows with empty data 
MegaTable= MegaTable[MegaTable["2000"]!=""]
#index reset
MegaTable= MegaTable.reset_index(drop=True)

#saving results
MegaTable.to_csv('MEGAtable.csv', index=False, header=False, encoding='utf-8')

In [187]:
#look at this beauty!
MegaTable

,Coords,Country,1995,2000,2005,2010,2011,2012,2013,2014,2015
0,"(30, 0)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
1,"(30, 2.07493358199)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
2,"(30, -2.07493358199)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
3,"(30, 4.14986716399)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
4,"(30, -4.14986716399)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
5,"(30, 6.22480074598)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
6,"(30, -6.22480074598)",Morocco,12.7,13.6,11,9.1,8.9,9,9.2,9.9,9.6
7,"(30, 8.29973432797)",Algeria,27.9,29.8,15.3,10,10,11,9.8,10.6,10.5
8,"(30, -8.29973432797)",Morocco,12.7,13.6,11,9.1,8.9,9,9.2,9.9,9.6
9,"(30, 10.37466791)",Libya,20.9,20.8,20.5,19.7,19.2,20.6,20.6,20.6,20.6


In [203]:
#testing - .json output  should be compatible with base code of p3
def buildJsonTest(df):
    result="["
    res1="[\"1990\",["
    for element in df.index:
        obj =df.ix[element]
        res1=res1+obj["Coords"][1:-1]+","+str(obj["1995"])+","
    res1=res1[:-1]
    res1+="]]"
    res2="[\"1995\",["
    for element in df.index:
        obj =df.ix[element]
        res2=res2+obj["Coords"][1:-1]+","+str(obj["2000"])+","
    res2=res2[:-1]
    res2+="]]"
    res3="[\"2000\",["
    for element in df.index:
        obj =df.ix[element]
        res3=res3+obj["Coords"][1:-1]+","+str(obj["2005"])+","
    res3=res3[:-1]
    res3+="]]"
    result+=res1+","+res2+","+res3+"]"
    output_file = open("population909500.json", "w")
    output_file.write(result)
    output_file.close()
    
#better method    
def buildJson(df):
    result="["
    years =["1995","2000","2005","2010","2011","2012","2013","2014","2015"]
    for year in years:
        res="[\"" + year + "\",["
        for element in df.index:
            obj =df.ix[element]
            res=res+obj["Coords"][1:-1]+","+str(obj[year])+","
        res=res[:-1]
        res+="]]"
        result+=res+","
    result=result[:-1]+"]"
    output_file = open("unemployment-1995-2015.json", "w")
    output_file.write(result)
    output_file.close()
        
    
    
    
    
    
    

In [204]:
#ok lets do it! after hours of brainstorming, bringing back my knowledge of pandas and  waiting geopy server responses!
buildJson(MegaTable)